In [1]:
import cvxpy as cp
import numpy as np
import osbdo as ob

# Hello world example
$$\begin{array}{ll}
\mbox{minimize } &\left((1/2)x_1^TP_1x_1+q_1^Tx_1+r_1\right)+\left(c_2^Tx_2+d_2\right)+\left((1/2)x_3^TP_3x_3+q_3^Tx_3+r_3\right)\\
\mbox{subject to }&l_1\leq x_1\leq u_1\\
&l_2\leq x_2\leq u_2\\
&l_3\leq x_3\leq u_3\\
&(x_{1})_1+2(x_2)_1=(x_3)_1
\end{array}$$

# Describe each $f_i$ in a class `Agent_i(osbdo.Agent)`

In [2]:
class AffineAgent(ob.Agent):
    """
       objective: c^T x + d
    """    
    def _construct_params(self):
        self.c = self.params['c']
        self.d = self.params['d']
        
    def query(self, *, v, solver):
        return ob.Point(x=v, q=self.c, f=(self.c.T@v+self.d).sum())
    
    def get_init_minorant(self):
        var = cp.Variable(self.dim)
        prob = cp.Problem(cp.Minimize(self.c.T@var+self.d),[var<=self.upb, var>=self.lwb])
        prob.solve()
        return cp.Constant(prob.value)

            
class QuadraticAgent(ob.Agent):
    """
       objective:(1/2) x^T P x + q^T x + r
    """
    def _construct_params(self):
        self.P = self.params['P']
        self.q = self.params['q']
        self.r = self.params['r']

    def query(self, *, v, solver):
        return ob.Point(x=v, q=self.P@v+self.q, f=(0.5*v.T@self.P@v+self.q.T@v+self.r).sum())
    
    def get_init_minorant(self):
        var = cp.Variable(self.dim)
        prob = cp.Problem(cp.Minimize(0.5*cp.quad_form(var,self.P)+self.q.T@var+self.r),\
                                                                    [var<=self.upb, var>=self.lwb])
        prob.solve()
        return cp.Constant(prob.value)

### Create 3 agents

The public variables have size $x_1  \in {\mathbf R}^5,~ x_2 \in {\mathbf R}^3,~ x_3 \in {\mathbf R}$.

In [3]:
# define agent 1
P = np.random.rand(5,5)
P1 = P.T@P
params1 = {'dimension':5, 'lower_bound':np.array([-2,-3,2,-6,-10]), 'upper_bound':np.array([1,3,6,9,-5]),
          'P': P1, 'q':np.random.rand(5), 'r':np.random.rand()}
tetiana = QuadraticAgent(params1)

# define agent 2
params2 = {'dimension':3, 'lower_bound':np.array([-2,6,-5]), 'upper_bound':np.array([5,10,-1]), 
          'c':np.random.rand(3), 'd':np.random.rand()}
fangzhao = AffineAgent(params2)

# define agent 3
P = np.random.rand(1,1)
P3 = P.T@P
params3 = {'dimension':1, 'lower_bound': np.array([-2]), 'upper_bound': np.array([17]),
          'P': P3, 'q': np.random.rand(1), 'r': np.random.rand()}
stephen = QuadraticAgent(params3)

In [4]:
agents = [tetiana, fangzhao, stephen]

# Coupling

In [5]:
domain = [tetiana.x[0] + 2 * fangzhao.x[0] == stephen.x]
g = ob.Coupling(agents=agents, \
                function = cp.Constant(0), domain = domain)

# Problem

In [6]:
prob = ob.Problem(agents = agents, g = g)

# Solve problem

In [7]:
x_agents, x_global = prob.solve()

k=0, rel_gap=9.619739370870892, L=3.353756015316975, U=35.61601479615676
k=1, rel_gap=9.616478973738262, L=3.354785977936684, U=35.61601479615676
k=2, rel_gap=5.835906987752071, L=3.3547859779496534, U=22.9330049090787
k=3, rel_gap=3.9505496955284602, L=3.354785977960471, U=16.608034701755358
k=4, rel_gap=2.9469276085295912, L=3.3547859779852405, U=13.241097397217892
k=5, rel_gap=2.0169321916419793, L=3.3547859779740903, U=10.121161813019153
k=6, rel_gap=1.4121526956072046, L=3.3547859782238296, U=8.092256040557864
k=7, rel_gap=0.9913308659523186, L=3.354785977944025, U=6.680488866543971
k=8, rel_gap=0.6278268648472287, L=3.3547859780312024, U=5.461010740851976
k=9, rel_gap=0.36316549678341775, L=3.354785977930203, U=4.573128494207269
k=10, rel_gap=0.19674531131619657, L=3.354785978007097, U=4.0148243896493145
k=11, rel_gap=0.10915528420371182, L=3.3547859779674587, U=3.720978594835124
k=12, rel_gap=0.061287345681784886, L=3.3547859779451072, U=3.560391905863834
k=13, rel_gap=0.0333754

# Centralized solution

In [8]:
x1 = cp.Variable(5)
x2 = cp.Variable(3)
x3 = cp.Variable(1)

f1 = 0.5 * cp.quad_form(x1,params1['P']) + params1['q'].T@x1 + params1['r']
f2 = params2['c'].T@x2 + params2['d']
f3 = 0.5 * cp.quad_form(x3,params3['P']) + params3['q'].T@x3 + params3['r']

constr1 = [x1 >= params1['lower_bound'], x1 <= params1['upper_bound']]
constr2 = [x2 >= params2['lower_bound'], x2 <= params2['upper_bound']]
constr3 = [x3 >= params3['lower_bound'], x3 <= params3['upper_bound']]
constr4 = [x1[0] + 2*x2[0] == x3]

cvx_prob = cp.Problem(cp.Minimize(f1+f2+f3), constr1+constr2+constr3+constr4)
cvx_prob.solve()

3.354785977927074

# Comparison

In [9]:
x_agents

[array([ 0.96034745,  2.88240132,  4.28726087, -0.63905436, -5.        ]),
 array([-1.48017372,  6.        , -5.        ]),
 array([-2.])]

In [10]:
x1.value, x2.value, x3.value

(array([ 1.        ,  2.81656147,  4.43089021, -0.77164592, -5.        ]),
 array([-1.5,  6. , -5. ]),
 array([-2.]))

# $L^k, \quad h^\star, \quad U^k$

In [11]:
prob.lower_bnd[-1], cvx_prob.value, prob.upper_bnd[-1]

(3.3547859782238296, 3.354785977927074, 3.38512111571562)